<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

# Scope and description

The data created by this notebook is an enriched version of the [`blobs` dataset](https://spatialdata.scverse.org/en/latest/generated/spatialdata.datasets.blobs.html).

Elements contained:
- 2D single-scale image (cyx);
- 2D multi-scale image (cyx);
- 2D single-scale labels (yx);
- 2D multi-scale labels (yx);
- 2D points;
- circles;
- polygons;
- multipolygons.

Annotations contained:
- table annotating the 2D labels.

Also the following is contained:
- `Identity`, `Scale`, `Translation`, `Affine`, `Sequence` transformations (for the `blobs_labels` element. Each transformation maps to a different coordinate system.

Note, this object is not meant to contain everything, example of features that are not covered by this notebook are:
- 3D images, labels, points
- table annotating multiple elements
- multiple tables
- dataframe elements containing annotation columns (we only have elements containing spatial information)

# Prepare the data

In [1]:
%load_ext jupyter_black

In [2]:
import warnings

# warnings clutter the diffs of notebooks
warnings.filterwarnings("ignore")

In [3]:
NAME = "multiple_elements"

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
from io_utils import delete_old_data, write_sdata_and_check_consistency
from spatialdata.datasets import blobs
import spatialdata_plot  # noqa: F401

delete_old_data(name=NAME)
sdata = blobs(length=64)
sdata

FileNotFoundError: [Errno 2] No such file or directory: 'multiple_elements.zarr/.zgroup'

In [ ]:
from spatialdata.transformations import (
    Affine,
    Identity,
    Scale,
    Sequence,
    Translation,
    get_transformation,
    set_transformation,
)

There is already a default coordinate system, called `global`, which is mapped using the `Identity` transformation.

In [ ]:
get_transformation(sdata["blobs_labels"], get_all=True)

Let's now add a coordinate system for each other transformation.

In [ ]:
scale = Scale([2, 3], axes=("x", "y"))
translation = Translation([10, -50], axes=("x", "y"))

transformations = {
    "global": Identity(),
    "scale": scale,
    "translation": translation,
    "affine": Affine(
        [[10, 20, 30], [40, 50, 60], [0, 0, 1]],
        input_axes=("x", "y"),
        output_axes=("x", "y"),
    ),
    "sequence": Sequence([scale, translation]),
}
set_transformation(sdata["blobs_labels"], transformations, set_all=True)

# Read-write and IO validation

In [ ]:
import logging

logging.getLogger("ome_zarr").setLevel(logging.ERROR)

write_sdata_and_check_consistency(sdata=sdata, name=NAME)

# Plot the data

## Single scale images, single scale labels, points

In [ ]:
sdata.pl.render_images("blobs_image").pl.render_labels("blobs_labels").pl.render_points(
    "blobs_points", na_color="w"
).pl.show(coordinate_systems="global")

## Labels for each coordinate system

In [ ]:
# we specify the coordinate systems otherwise the order is not deterministic
sdata.pl.render_labels("blobs_labels").pl.show(
    coordinate_systems=["global", "scale", "translation", "affine", "sequence"]
)

## labels data plotted for each coordinate system

## Lables annoted by the table (var)

In [ ]:
sdata["table"].obs

In [ ]:
sdata["table"].var_names

In [ ]:
sdata.pl.render_labels("blobs_labels", color="channel_0_sum").pl.show(coordinate_systems="global")

## Shapes: circles, polygons, multipolygons

In [ ]:
sdata.pl.render_shapes("blobs_circles", fill_alpha=0.5).pl.show(coordinate_systems="global")

In [ ]:
sdata.pl.render_shapes("blobs_polygons").pl.show(coordinate_systems="global")

In [ ]:
sdata.pl.render_shapes("blobs_multipolygons").pl.show(coordinate_systems="global")